In [1]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import RFECV
from sklearn.tree import DecisionTreeClassifier
from sklearn.decomposition import PCA
from sklearn import metrics
from skbio.diversity import beta_diversity
from skbio import TreeNode
from ete3 import NCBITaxa
from io import StringIO
import umap

In [2]:
from numpy import log, average, inf, nan, median, exp, interp, floor

In [3]:
def RLE_normalize(pd_dataframe):
    '''
    Normalize with Relative Log Expression
    INPUT:
        pd_dataframe(pandas DataFrame): Colums as Samples, Rows as OTUs
    OUTPUT:
        step7(pandas DataFrame): RLE Normalized. Colums as Samples, Rows as OTUs
    '''
    step1 = pd_dataframe.apply(log, 0)
    step2 = step1.apply(average, 1)
    step3 = step2[step2.replace([inf, -inf], nan).notnull()]
    step4_1 = step1[step1.replace(
        [inf, -inf], nan).notnull().all(axis=1)]
    step4 = step4_1.subtract(step3, 0)
    step5 = step4.apply(median, 0)
    step6 = step5.apply(exp)
    step7 = pd_dataframe.divide(step6, 1).apply(round, 1)
    return(step7)

In [113]:
class sourcemap():
    def __init__(self, train, test, labels, norm_method):
        '''
        train(pandas DataFrame) source otu table
        test(pandas DataFrame) sink otu table
        labels(list) train sample class
        norm_method(str) normalization method
        '''
        self.train = pd.read_csv(train, index_col=0)
        self.test = pd.read_csv(test, index_col=0)
        self.comb = self.train.merge(
            self.test, how='outer', left_index=True, right_index=True).fillna(0)
        if norm_method == 'RLE':
            self.combined = RLE_normalize(self.comb).T
        self.train_samples = list(self.train.columns)
        self.test_samples = list(self.test.columns)
        labels = pd.read_csv(labels, index_col=0)
        self.labels = labels['labels']

    def compute_distance(self, rank='species'):
        # Getting a single Taxonomic rank
        ncbi = NCBITaxa()
        only_rank = []
        for i in list(self.combined.columns):
            try:
                if ncbi.get_rank([i])[i] == rank:
                    only_rank.append(i)
            except KeyError:
                continue
        self.normalized_rank = self.combined.loc[:,only_rank]
        tree = ncbi.get_topology(
            list(self.normalized_rank.columns), intermediate_nodes=False)
        newick = TreeNode.read(StringIO(tree.write()))
        wu = beta_diversity("weighted_unifrac", self.normalized_rank.as_matrix().astype(int), ids=list(
            self.normalized_rank.index), otu_ids=[str(i) for i in list(self.normalized_rank.columns)], tree=newick)
        self.wu = wu.to_data_frame()

    def embed(self, umap_csv, n_comp=200):
        my_umap = umap.UMAP(metric='precomputed',
                            n_neighbors=30, min_dist=0.03, n_components=n_comp, n_epochs=500)
        umap_embed_a = my_umap.fit(self.wu)
        cols = [f"PC{i}" for i in range(1, n_comp+1)]
        self.umap = pd.DataFrame(
            umap_embed_a.embedding_, columns=cols, index=self.normalized_rank.index)

        if umap_csv:
            to_write = self.umap.copy(deep=True)
            y = self.labels.copy(deep=True)
            y = y.append(
                pd.Series(data=['sink']*len(list(self.test.index)), index=self.test.index))
            to_write['label'] = y
            to_write['name'] = to_write.index
            to_write.to_csv(umap_csv)

        self.source = self.umap.drop(self.test_samples, axis=0)
        self.source['label'] = self.labels
        self.sink = self.umap.drop(self.train_samples, axis=0)

    def knn_classification(self, kfold, threads, seed):
        train_features, test_features, train_labels, test_labels = train_test_split(
            self.source.drop('label', axis=1), self.source.loc[:, 'label'], test_size=0.2, random_state=seed)
        knn = KNeighborsClassifier(n_jobs=threads)

        param_knn_grid = {
            'n_neighbors': [3, 5, 10, 15, 20],
            'weights': ['uniform', 'distance']
        }

        CV_knn = GridSearchCV(
            estimator=knn, param_grid=param_knn_grid, cv=kfold, n_jobs=threads)

        print(
            f"\tPerforming {kfold} fold cross validation on {threads} cores...")
        CV_knn.fit(train_features, train_labels)

        knn1 = KNeighborsClassifier(
            n_neighbors=CV_knn.best_params_['n_neighbors'], weights=CV_knn.best_params_['weights'], n_jobs=threads)

        knn1.fit(train_features, train_labels)
        y_pred = knn1.predict(test_features)
        print("\t-> Testing Accuracy:", metrics.accuracy_score(
            test_labels, y_pred))
        self.sink_pred = knn1.predict_proba(self.sink)
        print_class(samples =self.sink.index, classes=knn1.classes_, pred=self.sink_pred)
        predictions = class2dict(
            samples = self.sink.index, classes=knn1.classes_, pred=self.sink_pred)
        return(predictions)

In [114]:
def print_class(samples, classes, pred):
    print("\t----------------------")
    for i in range(0, len(samples)):
        sample = samples[i]
        print(f"\t- Sample: {sample}")
        [print(f'\t\t {i}:{j}')
         for i, j in zip(list(classes), list(pred[i, :]))]

In [115]:
def class2dict(samples, classes, pred):
    resdict = {}
    for i in range(0, len(samples)):
        sample = samples[i]
        resdict[sample] = {c: float(p) for (c, p) in zip(
            list(classes), list(pred[i, :]))}
    return(resdict)

In [116]:
u = sourcemap(train=mysource, test=mysink, labels = mylabels, norm_method = "RLE")

In [117]:
u.compute_distance(rank="species")

/projects1/users/borry/15_miniconda3/envs/sourcepredict/lib/python3.6/site-packages/ipykernel_launcher.py:34: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.


In [118]:
u.embed(n_comp=2, umap_csv="test.csv")

In [119]:
predicted_source = u.knn_classification(
        kfold=2, threads=2, seed=2)

	Performing 2 fold cross validation on 2 cores...
	-> Testing Accuracy: 0.9861111111111112
	----------------------
	- Sample: ERR1915662
		 Canis_familiaris:1.0
		 Industrialized_humans:0.0
		 Non_industrialized_humans:0.0
		 Sus_scrofa:0.0
	- Sample: ERR1915662_copy
		 Canis_familiaris:1.0
		 Industrialized_humans:0.0
		 Non_industrialized_humans:0.0
		 Sus_scrofa:0.0


In [120]:
predicted_source

{'ERR1915662': {'Canis_familiaris': 1.0,
  'Industrialized_humans': 0.0,
  'Non_industrialized_humans': 0.0,
  'Sus_scrofa': 0.0},
 'ERR1915662_copy': {'Canis_familiaris': 1.0,
  'Industrialized_humans': 0.0,
  'Non_industrialized_humans': 0.0,
  'Sus_scrofa': 0.0}}